In [ ]:
!pip install -q python-terrier
import pyterrier as pt
if not pt.started():
    pt.init()

# from pyterrier.measures import *
# dataset = pt.get_dataset('msmarco_passage')

In [ ]:
def msmarco_generate():
    dataset = pt.get_dataset("msmarco_passage")
    with pt.io.autoopen(dataset.get_corpus()[0], 'rt') as corpusfile:
        for l in corpusfile:
            docno, passage = l.split("\t")
            yield {'docno' : docno, 'text' : passage}

iter_indexer = pt.IterDictIndexer("./passage_index")
indexref = iter_indexer.index(msmarco_generate(), meta={})

BM25_withFeatures = pt.FeaturesBatchRetrieve(indexref, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2", "WMODEL:TF_IDF"])

import xgboost as xgb
params = {'objective': 'rank:ndcg', 
          'learning_rate': 0.1, 
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6,
          'verbose': 2,
          'random_state': 42 
         }

xgb_pipe= xgb.sklearn.XGBRanker(**params)



new_xgb_pipe = BM25_withFeatures >> pt.ltr.apply_learned_model(xgb_pipe,ltr)


new_xgb_pipe.fit(dataset.get_topics('train'), dataset.get_qrels('train'), dataset.get_topics('dev'), dataset.get_qrels('dev'))

results = pt.Experiment([BM25_withFeatures, 
                        new_xgb_pipe ],
                       dataset.get_topics("test-2019"), 
                       dataset.get_qrels("test-2019"),
                       eval_metrics=["ndcg_cut_10","map"],
                       names=["BM25", "xgBoost" ]
                  )
print(results)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eeb7fcb4-84cb-4f4a-9830-9d1fcbc47e1d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>